In [1]:
# download hansard from https://www.parlimen.gov.my/hansard-dewan-rakyat.html?uweb=dr
# !wget https://dlcdn.apache.org/tika/2.6.0/tika-server-standard-2.6.0-bin.zip
# !unzip tika-server-standard-2.6.0-bin.zip
# !chmod +x tika-server-standard-2.6.0-bin/bin/tika
# !./tika-server-standard-2.6.0-bin/bin/tika start -p 9998
# !pip3 install tika

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import tika
from tika import parser
import re
from unidecode import unidecode
from tqdm import tqdm
from glob import glob
from datetime import datetime
import json
from bs4 import BeautifulSoup

def cleaning(string):
    string = unidecode(string).replace('\t', ' ').replace('\r', ' ').replace('\n', ' ').replace('_', ' ')
    string = string.replace(' -', '-').replace(' ANYA', 'ANYA').replace('ki ta', 'kita').replace('s aya', 'saya')
    string = string.replace('m enjadi', 'menjadi').replace('meno lak', 'menolak')
    try:
        soup = BeautifulSoup(string, 'lxml')
        string = soup.text
    except:
        pass
    return re.sub(r'[ ]+', ' ', string).strip()

In [4]:
files = sorted(glob('pdf/*.pdf'))
files[0]

'pdf/DR-01032000.pdf'

In [5]:
files[0]

'pdf/DR-01032000.pdf'

In [34]:
raw_xml = parser.from_file('pdf/DR-04112003.pdf', 'http://localhost:9998/tika', xmlContent=True)
body = raw_xml['content'].split('<body>')[1].split('</body>')[0]
body_without_tag = body.replace("<p>", "").replace("</p>", "").replace("<div>", "").replace("</div>","").replace("<p />","")
text_pages = body_without_tag.split("""<div class="page">""")[1:]

In [35]:
text_pages[87]

'\n76 DR 04.11.2003 \n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n\n'

In [20]:
splitted = text_pages[87].split('\n \n')
splitted = [s for s in splitted if len(s.strip()) > 1]
if len(splitted) < 3:
    splitted = t.split('\n\n')
splitted = [cleaning(s) for s in splitted[1:]]
splitted = [s + '.' if s[-1] not in  '.;:,' else s for s in splitted if len(s)]
splitted = ' '.join(splitted)
len(splitted)

3790

In [36]:
with open('hansard.jsonl', 'w') as fopen:
    for f in tqdm(files):
        raw_xml = parser.from_file(f, 'http://localhost:9998/tika', xmlContent=True)
        body = raw_xml['content'].split('<body>')[1].split('</body>')[0]
        body_without_tag = body.replace("<p>", "").replace("</p>", "").replace("<div>", "").replace("</div>","").replace("<p />","")
        text_pages = body_without_tag.split("""<div class="page">""")[1:]

        for i, t in enumerate(text_pages):
            r = re.findall(r'DR[\. ]\s*[0-3]?[0-9].[0-3]?[0-9].(?:[0-9]{2})?[0-9]{2}\s+\d+\b', t)
            r_ = re.findall(r'\d+\s+DR[\. ]\s*[0-3]?[0-9].[0-3]?[0-9].(?:[0-9]{2})?[0-9]{2}\b', t)
            found = True
            if len(r):
                no_page = r[0].split()[-1]
            elif len(r_):
                no_page = r_[0].split()[0]
            else:
                found = False

            if not found:
                continue

            splitted = t.split('\n \n')
            splitted = [s for s in splitted if len(s.strip()) > 1]
            if len(splitted) < 3:
                splitted = t.split('\n\n')
            splitted = [cleaning(s) for s in splitted[1:]]
            splitted = [s + '.' if s[-1] not in  '.;:,' else s for s in splitted if len(s)]
            splitted = ' '.join(splitted)
            date = datetime.strptime(f, 'pdf/DR-%d%m%Y.pdf').strftime('%Y-%m-%d')
            
            if not len(splitted):
                print(f, i, t)

            d = {
                'original': t,
                'cleaned': splitted,
                'no_page': int(no_page),
                'actual_no_page': i + 1,
                'date': date,
                'url': f'https://www.parlimen.gov.my/files/hindex/{f}'
            }
            fopen.write(f'{json.dumps(d)}\n')
#             break
#         break

 11%|█████████▋                                                                            | 175/1560 [00:59<07:22,  3.13it/s]

pdf/DR-04112003.pdf 87 
76 DR 04.11.2003 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 





 14%|███████████▋                                                                          | 213/1560 [01:11<05:01,  4.47it/s]

pdf/DR-05102005.pdf 96 
DR 5.10.2005 83 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 





 23%|███████████████████▌                                                                  | 354/1560 [01:57<05:08,  3.91it/s]

...

 97%|██████████████████████████████████████████████████████████████████████████████████▏  | 1508/1560 [08:13<00:19,  2.62it/s]

pdf/DR-30082006.pdf 180 
DR.30.8.2006 165 

 

 

 

 

 

 

 

 





100%|████████████████████████████████████████████████████████████████████████████████████▊| 1557/1560 [08:30<00:01,  2.96it/s]

pdf/DR-31102016.pdf 148 
DR 31.10.2016                                                                                                                                                   137               

 

 

 

 

 

 

 





100%|█████████████████████████████████████████████████████████████████████████████████████| 1560/1560 [08:32<00:00,  3.05it/s]


In [37]:
!wc -l hansard.jsonl

142609 hansard.jsonl


In [38]:
!split -l 50000 -d --additional-suffix=.splitted hansard.jsonl hansard.jsonl